# Exploratory Data Analysis of Engineered Features

Observations:
* Moderate to weak visual correlations between engineered features and targets
* Score of first assessment looks to have a sigificant collelation with estimated final score, final outcome
* Individual distributions show a lot of 0 values, unsurprising otherwise

In [1]:
import numpy as np 
import pandas as pd
import seaborn as sns
sns.reset_defaults
sns.set_style(style='whitegrid')
sns.set_context(context='poster')
import matplotlib.pyplot as plt
# plt.style.use('ggplot')
font = {'size'   : 16}
plt.rc('font', **font)
plt.ion()
%matplotlib inline
plt.rcParams["patch.force_edgecolor"] = True
plt.rcParams['figure.figsize'] = (20.0, 10.0)
pd.set_option('display.max_columns', 2000)
pd.set_option('display.max_rows', 2000)# reg_df = pd.read_csv('../data/raw/studentRegistration.csv')
# reg_df = pd.read_csv('../data/raw/studentRegistration.csv')


In [2]:
df = pd.read_csv('../data/processed/transformed_data_with_features.csv')

In [ ]:
df.columns

In [3]:
df.shape

(32630, 63)

In [11]:
df[(df['date_unregistration'].isnull())].shape

(0, 63)

In [6]:
df[(df['date_unregistration'] > 0) | (df['date_unregistration'].isnull())].shape

(6979, 63)

In [ ]:
df.fillna(value=0, inplace=True)

In [ ]:
X_train = pd.read_csv('../data/processed/X_train.csv')

In [ ]:
X_train.columns

In [ ]:
df.corr()

In [ ]:
df.columns

## Engineered Feature Analysis

### Bivariate Plots Against Targets: final_result_num, estimated_final_score

In [ ]:
sns.jointplot(x='clicks_per_day', y='final_result_num', data=df, size=12, alpha=0.2)

In [ ]:
sns.jointplot(x='clicks_per_day', y='estimated_final_score', data=df, size=12, alpha=0.2)

In [ ]:
sns.jointplot(x='pct_days_vle_accessed', y='final_result_num', data=df, size=12, alpha=0.2)

In [ ]:
sns.jointplot(x='pct_days_vle_accessed', y='estimated_final_score', data=df, size=12, alpha=0.2)

In [ ]:
sns.jointplot(x='studied_credits', y='final_result_num', data=df, size=12, alpha=0.2)

In [ ]:
sns.jointplot(x='studied_credits', y='estimated_final_score', data=df, size=12, alpha=0.2)

In [ ]:
df.columns

In [ ]:
sns.jointplot(x='max_clicks_one_day', y='final_result_num', data=df, size=12, alpha=0.5)

In [ ]:
sns.jointplot(x='max_clicks_one_day', y='estimated_final_score', data=df, size=12, alpha=0.2)

In [ ]:
sns.jointplot(x='first_date_vle_accessed', y='final_result_num', data=df, size=12, alpha=0.2)

In [ ]:
sns.jointplot(x='first_date_vle_accessed', y='estimated_final_score', data=df, size=12, alpha=0.2)

In [ ]:
sns.jointplot(x='avg_days_sub_early', y='final_result_num', data=df, size=12, alpha=0.2)

In [ ]:
sns.jointplot(x='avg_days_sub_early', y='estimated_final_score', data=df, size=12, alpha=0.2)

In [ ]:
df.columns

In [ ]:
sns.jointplot(x='days_early_first_assessment', y='final_result_num', data=df, size=12, alpha=0.2)

In [ ]:
sns.jointplot(x='days_early_first_assessment', y='estimated_final_score', data=df, size=12, alpha=0.2)

In [ ]:
sns.jointplot(x='score_first_assessment', y='final_result_num', data=df, size=12, alpha=0.2)

In [ ]:
sns.jointplot(x='score_first_assessment', y='estimated_final_score', data=df, size=12, alpha=0.2)

In [ ]:
sns.jointplot(x='avg_score', y='final_result_num', data=df, size=12, alpha=0.2)

In [ ]:
sns.jointplot(x='avg_score', y='estimated_final_score', data=df, size=12, alpha=0.1)

### Univariate Plots of Engineered Features

In [ ]:
df.columns

In [ ]:
f = list(df.columns[[6,7,8,9,10,11,12,13,14]])

In [ ]:
for feat in f:
    plt.figure(figsize=(12,12))
    sns.distplot(df[feat], bins=50, kde=False)